In [4]:
import os
import sys

# 将工作目录切换到 clean.py 所在的目录
os.chdir(r'C:\Users\ASUS\TensorFlow Keywords\clean-text-main\cleantext')

# 将当前工作目录添加到 Python 解释器的搜索路径
sys.path.append(r'C:\Users\ASUS\TensorFlow Keywords\clean-text-main\cleantext')

from clean import clean

data_dir = r"C:\Users\ASUS\TensorFlow Keywords\test"

# 创建一个新的目录来保存清洗后的文本数据
clean_data_dir =  r"C:\Users\ASUS\TensorFlow Keywords\test_clean"
os.makedirs(clean_data_dir, exist_ok=True)

# 遍历目标地址中的所有文件夹及其子目录
for root, dirs, files in os.walk(data_dir):
    for file_name in files:
        if not file_name.startswith('.'):  # 排除隐藏文件
            file_path = os.path.join(root, file_name)
            # 读取文件内容
            with open(file_path, "r", encoding="latin1") as file:
                text = file.read()
            # 对文本进行清洗
            clean_text = clean(text, no_line_breaks=True, fix_unicode=True, to_ascii=True, lower=True, no_urls=True, no_emails=True, no_phone_numbers=True, no_numbers=True, no_digits=True, no_currency_symbols=True, no_punct=True, no_emoji=True, remove_stopwords=True)
            # 构建清洗后的文件路径
            relative_path = os.path.relpath(file_path, data_dir)
            clean_file_path = os.path.join(clean_data_dir, relative_path)
            os.makedirs(os.path.dirname(clean_file_path), exist_ok=True)
            # 将清洗后的文本保存到新文件中
            with open(clean_file_path, "w", encoding="utf-8") as clean_file:
                clean_file.write(clean_text)


In [5]:
import sys
sys.path.append(r'C:\Users\ASUS\TensorFlow Keywords\CBOW_master')
import os
import nltk
from cbow_model import CBOW
from tqdm import tqdm

# 读取文本数据
input_folder = 'C:/Users/ASUS/TensorFlow Keywords/test_clean/'

# 创建 word_vector 文件夹以保存模型文件
output_model_folder = 'C:/Users/ASUS/TensorFlow Keywords/test_vector'
os.makedirs(output_model_folder, exist_ok=True)

# 遍历每个子文件夹中的所有文件
for category in os.listdir(input_folder):
    category_path = os.path.join(input_folder, category)
    if os.path.isdir(category_path):
        for file_name in os.listdir(category_path):
            file_path = os.path.join(category_path, file_name)
            if os.path.isfile(file_path):
                # 读取文件内容
                with open(file_path, "r", encoding="utf-8") as file:
                    text = file.read()

                # 创建 CBOW 模型实例
                model = CBOW(text=text)

                # 设置训练参数
                window = 2
                n = 100
                learning_rate = 0.0001

                # 分割文本
                split_text = model.text.lower().split()

                # 训练模型
                for center_word in tqdm(split_text, desc='Training', unit='word'):
                    index = split_text.index(center_word)
                    left_context = split_text[max(0, index - window):index]
                    right_context = split_text[index + 1:min(len(split_text), index + window + 1)]
                    avg_vector = model.get_average_vector_context(left_context, right_context)
                    x_hat = model.get_score(avg_vector)
                    y_hat = model.softmax(x_hat)
                    center_word_index = model.word_index[center_word]
                    center_word_one_hot_encoding = model.get_one_hot(center_word_index)
                    error = model.compute_cross_entropy_error(center_word_one_hot_encoding, y_hat)
                    model.update_U(error, avg_vector)
                    model.update_V(error, left_context, right_context)

                # 保存模型文件
                output_model_file = os.path.join(output_model_folder, f"{file_name}.bin")
                model.save_model(output_model_file)

Training:  40%|███████████████████████████                                         | 39/98 [00:00<00:00, 2675.99word/s]


KeyboardInterrupt: 